In [28]:
file_path = '../files/midi/Montgomery,_Wes_-_West_Coast_Blues_gp3/test.mid'
info_path = '../files/audio/Montgomery,_Wes_-_West_Coast_Blues_gp3/info.json'
import mido
import json
import numpy as np

In [7]:
song = mido.MidiFile(file_path)
with open(info_path, 'r') as fp:
        info_dictionary = json.load(fp)

In [8]:
print(info_dictionary)

{'track_0': {'type': 'guitar'}, 'track_1': {'type': 'guitar'}, 'track_2': {'type': 'guitar'}, 'track_3': {'type': 'bass'}, 'track_4': {'type': 'guitar'}, 'track_5': {'type': 'percussion'}}


In [33]:
for i_track, track in enumerate(song.tracks):
    tmp_track_name = info_dictionary['track_' + str(i_track)]['type']
    print(tmp_track_name)
    if tmp_track_name == 'bass':
        note_ons = -1*np.ones(127).astype(int)
        notes_on_offs = []
        note_on_num = 0
        # count how many notes in the bass channel
        for i_message, message in enumerate(track):
            if message.type == 'note_on':
                if message.velocity > 0:
                    # keep the index that triggered the note on
                    note_ons[message.note] = note_on_num
                    tmp_note_obj = {'on_idx': i_message, 'off_idx': -1, 'pitch': message.note}
                    notes_on_offs.append( tmp_note_obj )
                    note_on_num += 1
                else:
                    if note_ons[message.note] > -1:
                        notes_on_offs[ note_ons[message.note] ]['off_idx'] = i_message
                        note_ons[message.note] = -1
                    else:
                        print('ERROR: note off without previous note one')

guitar
guitar
guitar
bass
guitar
percussion


In [32]:
print(notes_on_offs)

[{'on_idx': 8, 'off_idx': 9, 'pitch': 34}, {'on_idx': 10, 'off_idx': 11, 'pitch': 41}, {'on_idx': 12, 'off_idx': 13, 'pitch': 44}, {'on_idx': 14, 'off_idx': 15, 'pitch': 51}, {'on_idx': 16, 'off_idx': 17, 'pitch': 56}, {'on_idx': 18, 'off_idx': 19, 'pitch': 34}, {'on_idx': 20, 'off_idx': 21, 'pitch': 41}, {'on_idx': 22, 'off_idx': 23, 'pitch': 41}, {'on_idx': 24, 'off_idx': 25, 'pitch': 47}, {'on_idx': 26, 'off_idx': 27, 'pitch': 42}, {'on_idx': 28, 'off_idx': 29, 'pitch': 35}, {'on_idx': 30, 'off_idx': 31, 'pitch': 40}, {'on_idx': 32, 'off_idx': 34, 'pitch': 47}, {'on_idx': 33, 'off_idx': 35, 'pitch': 34}, {'on_idx': 36, 'off_idx': 37, 'pitch': 41}, {'on_idx': 38, 'off_idx': 39, 'pitch': 39}, {'on_idx': 40, 'off_idx': 41, 'pitch': 39}, {'on_idx': 42, 'off_idx': 43, 'pitch': 46}, {'on_idx': 44, 'off_idx': 45, 'pitch': 44}, {'on_idx': 46, 'off_idx': 47, 'pitch': 38}, {'on_idx': 48, 'off_idx': 49, 'pitch': 45}, {'on_idx': 50, 'off_idx': 51, 'pitch': 43}, {'on_idx': 52, 'off_idx': 53, 'pi